# Tutorial: Batch Flight Requests
The `google_flights_scraper` package can also be used to make batch request of multiple flights where a certain criteria is varied (e.g. destination or dates)

## Set Up

In [4]:
# Import
from datetime import datetime, timedelta

import pandas as pd

from google_flights_scraper.jupyter_helper import (
    scrape_multiple,
)

pd.set_option("display.max_columns", None)

# Get today's date
today = datetime.today()

## Multiple Searches
Search multiple destinations and dates. Helpful if you know you want to travel, but don't know where or when and are looking for good deals. 


Below are the possible parameters when making a request:  

**Required Inputs**

`departure_code` : Departure airport or city (a single location)  
`departure_country` Departure country (a single country)  
`arrival_codes` : A list of possible destination airports or cities  
`arrival_countries` : The matching list of possible destination countries  
`start_dates` : A list of possible departure dates (in `"MM/DD/YYY"` format)  
`end_dates` : A list of possible arrrival dates  (in `"MM/DD/YYY"` format)  
`seat_classes` : The matching list of seat classes (reason this is `list` and not `str` is because domestic and international flights can have different class names)  

**Optional Inputs**  

`output_path` : path to output `.csv`  (ex: `export.csv`)  
`delay_seconds`: How many seconds to delay between requests. Can be helpful to increase if you are being rate limited by Google.    
`delay_jitter`: How many seconds to jitter delay_seconds
`n_jobs`: Number of concurrent scrapes (default 1). When >1, `delay_seconds` is ignored and concurrency is controlled by the semaphore. 
`task_timeout`: Max seconds to wait for a single scrape before cancelling 

### Example: Departing from New York

In this example, we are departing from New York and searching for flights to SEA, London, Paris, and SFO.

In [2]:
# Search Mapping
city_search = {
    "SEA": {"country": "United States of America", "seat_class": "Economy (exclude Basic)"},
    "London": {"country": "United Kingdom", "seat_class": "Economy"},
    "Paris": {"country": "France", "seat_class": "Economy"},
    "SFO": {"country": "United States of America", "seat_class": "Economy (exclude Basic)"},
}

# Individual Lists
destination_codes = list(city_search.keys())
destination_countries = [city_search[code]["country"] for code in destination_codes]
seat_classes = [city_search[code]["seat_class"] for code in destination_codes]

# Create Dates
starts = [(today + timedelta(weeks=8)).strftime("%m/%d/%Y")] * 4
ends = [(today + timedelta(weeks=10)).strftime("%m/%d/%Y")] * 4

# Run
df_destinations = scrape_multiple(
    departure_code="New York",
    departure_country="United States of America",
    arrival_codes=destination_codes,
    arrival_countries=destination_countries,
    start_dates=starts,
    end_dates=ends,
    seat_classes=seat_classes,
    delay_seconds=3.0,
    n_jobs=4,
)

In [5]:
df_destinations

,departure_airport,departure_country,arrival_airport,arrival_country,departure_date,return_date,seat_class,departure_airline,departure_departure_airport,departure_departure_date,departure_departure_time,departure_num_stops,departure_connection_airports,departure_layover_durations,departure_layover_total_minutes,departure_arrival_airport,departure_arrival_date,departure_arrival_time,departure_duration_minutes,departure_duration_str,departure_carry_on_bags,departure_checked_bags,return_airline,return_departure_airport,return_departure_date,return_departure_time,return_num_stops,return_connection_airports,return_layover_durations,return_layover_total_minutes,return_arrival_airport,return_arrival_date,return_arrival_time,return_duration_minutes,return_duration_str,return_carry_on_bags,return_checked_bags,price,price_classification,price_difference,price_relativity,status,url
0,New York,United States of America,Paris,France,04/08/2026,04/22/2026,Economy,Icelandair,Newark Liberty International Airport,"Wednesday, April 8",8:25 PM,1,[Keflavík International Airport],[1 hr 20 min],80,Paris Charles de Gaulle Airport,"Thursday, April 9",1:00 PM,635,10 hr 35 min,NaN,NaN,Icelandair,Paris Charles de Gaulle Airport,"Wednesday, April 22",2:00 PM,1,[Keflavík International Airport],[1 hr 25 min],85,Newark Liberty International Airport,"Wednesday, April 22",7:15 PM,675,11 hr 15 min,NaN,NaN,316,low,589,0.6508,Ran successfully.,https://www.google.com/travel/flights/booking?...
1,New York,United States of America,SFO,United States of America,04/08/2026,04/22/2026,Economy (exclude Basic),JetBlue,John F. Kennedy International Airport,"Wednesday, April 8",5:20 PM,1,[Boston Logan International Airport],[2 hr 27 min],147,San Francisco International Airport,"Thursday, April 9",12:56 AM,636,10 hr 36 min,1.0,0.0,JetBlue,San Francisco International Airport,"Wednesday, April 22",2:45 PM,0,[],[],0,John F. Kennedy International Airport,"Wednesday, April 22",11:29 PM,344,5 hr 44 min,1.0,0.0,311,low,186,0.3742,Ran successfully.,https://www.google.com/travel/flights/booking?...
2,New York,United States of America,SEA,United States of America,04/08/2026,04/22/2026,Economy (exclude Basic),United,Newark Liberty International Airport,"Wednesday, April 8",7:50 AM,0,[],[],0,Seattle-Tacoma International Airport,"Wednesday, April 8",11:02 AM,372,6 hr 12 min,1.0,0.0,United,Seattle-Tacoma International Airport,"Wednesday, April 22",12:08 PM,0,[],[],0,Newark Liberty International Airport,"Wednesday, April 22",8:29 PM,321,5 hr 21 min,1.0,0.0,319,low,183,0.3645,Ran successfully.,https://www.google.com/travel/flights/booking?...
3,New York,United States of America,London,United Kingdom,04/08/2026,04/22/2026,Economy,American,John F. Kennedy International Airport,"Wednesday, April 8",7:20 PM,0,[],[],0,Heathrow Airport,"Thursday, April 9",7:20 AM,420,7 hr,NaN,NaN,Finnair,Heathrow Airport,"Wednesday, April 22",10:20 AM,1,[Helsinki Airport],[2 hr],120,John F. Kennedy International Airport,"Wednesday, April 22",7:10 PM,830,13 hr 50 min,NaN,NaN,514,low,112,0.1789,Ran successfully.,https://www.google.com/travel/flights/booking?...


This returns a pandas DataFrame of the best flight for each destination, sorted in order of `price_relativity` highlight the flights which are the "best deal".

This also contains all the other information returned by the basic flight request function.